<a href="https://colab.research.google.com/github/asheta66/CNN/blob/main/Bridge_Cracks/Bridge_Cracks_Detection_Using_TL_3_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**How to use it in Colab:**

1. Run Cells 1–4 once to prepare your environment and datasets.

2. Run Cell 5, 6, 7 separately for each model.

3. Run Cells 8–9 after training a model to generate editable figures and save them.

**Cell 1: Mount Google Drive & Imports**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetB0, ResNet50, MobileNetV2
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, roc_curve, auc
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os

AUTOTUNE = tf.data.AUTOTUNE


Mounted at /content/drive


**Cell 2: Dataset Preparation**

In [2]:
# Dataset path
dataset_dir = '/content/drive/MyDrive/Bridge_Cracks'

# Classes
class_names = sorted(os.listdir(dataset_dir))
class_to_label = {name: idx for idx, name in enumerate(class_names)}
num_classes = len(class_names)
print("Classes:", class_names)

# Collect image paths and labels
image_paths, labels = [], []
for class_name in class_names:
    class_path = os.path.join(dataset_dir, class_name)
    for fname in os.listdir(class_path):
        image_paths.append(os.path.join(class_path, fname))
        labels.append(class_to_label[class_name])

image_paths = np.array(image_paths)
labels = np.array(labels)

# Fixed image size
IMG_HEIGHT, IMG_WIDTH = 224, 224
print("Image size:", IMG_HEIGHT, "x", IMG_WIDTH)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    image_paths, labels, test_size=0.2, stratify=labels, random_state=42
)


Classes: ['Negative', 'Positive']
Image size: 224 x 224


**Cell 3: Data Augmentation & Dataset Loader**

In [3]:
# Data augmentation
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
])

# Dataset loader
def load_dataset(paths, labels, training=True, batch_size=32):
    ds = tf.data.Dataset.from_tensor_slices((paths, labels))

    def process(path, label):
        img = tf.io.read_file(path)
        img = tf.image.decode_jpeg(img, channels=3)
        img = tf.image.resize(img, (IMG_HEIGHT, IMG_WIDTH))
        img = tf.cast(img, tf.float32) / 255.0
        return img, label

    ds = ds.map(process, num_parallel_calls=AUTOTUNE)

    if training:
        ds = ds.map(lambda x, y: (data_augmentation(x, training=True), y), num_parallel_calls=AUTOTUNE)
        ds = ds.shuffle(1000)

    return ds.batch(batch_size).prefetch(AUTOTUNE)

# Load datasets
train_ds = load_dataset(X_train, y_train, training=True)
test_ds  = load_dataset(X_test, y_test, training=False)


**Cell 4: Model Builder**

In [4]:
def build_model(base_model_fn):
    base_model = base_model_fn(weights='imagenet', include_top=False,
                               input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))
    base_model.trainable = True

    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])

    model.compile(
        optimizer=tf.keras.optimizers.Adam(1e-4),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model


**Cell 5: Train EfficientNetB0**

In [ ]:
model_name = "EfficientNetB0"
model_fn = EfficientNetB0

print(f"Training {model_name}")
model = build_model(model_fn)

EPOCHS = 10
history = model.fit(train_ds, validation_data=test_ds, epochs=EPOCHS)

# Save model
model.save(f"/content/drive/MyDrive/{model_name}_bridge_cracks.h5")


Training EfficientNetB0
16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/10
95/95 ━━━━━━━━━━━━━━━━━━━━ 883s 8s/step - accuracy: 0.7978 - loss: 0.4290 - val_accuracy: 0.7293 - val_loss: 0.5812
Epoch 2/10
71/95 ━━━━━━━━━━━━━━━━━━━━ 2:54 7s/step - accuracy: 0.9538 - loss: 0.1248

**Cell 6: Train ResNet50**

In [ ]:
model_name = "ResNet50"
model_fn = ResNet50

print(f"Training {model_name}")
model = build_model(model_fn)

EPOCHS = 10
history = model.fit(train_ds, validation_data=test_ds, epochs=EPOCHS)

# Save model
model.save(f"/content/drive/MyDrive/{model_name}_bridge_cracks.h5")


**Cell 7: Train MobileNetV2**

In [ ]:
model_name = "MobileNetV2"
model_fn = MobileNetV2

print(f"Training {model_name}")
model = build_model(model_fn)

EPOCHS = 10
history = model.fit(train_ds, validation_data=test_ds, epochs=EPOCHS)

# Save model
model.save(f"/content/drive/MyDrive/{model_name}_bridge_cracks.h5")


**Cell 8: Plot Confusion Matrices**

In [ ]:
# Predictions
y_train_true = np.concatenate([y for x, y in train_ds], axis=0)
y_test_true  = np.concatenate([y for x, y in test_ds], axis=0)

y_train_prob = model.predict(train_ds)
y_test_prob  = model.predict(test_ds)

y_train_pred = np.argmax(y_train_prob, axis=1)
y_test_pred  = np.argmax(y_test_prob, axis=1)

# Confusion matrices
cm_train = confusion_matrix(y_train_true, y_train_pred)
cm_test  = confusion_matrix(y_test_true, y_test_pred)

plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
sns.heatmap(cm_train, annot=True, fmt="d", cmap="Blues",
            xticklabels=class_names, yticklabels=class_names)
plt.title(f"{model_name} – Training CM")
plt.xlabel("Predicted")
plt.ylabel("True")

plt.subplot(1,2,2)
sns.heatmap(cm_test, annot=True, fmt="d", cmap="Blues",
            xticklabels=class_names, yticklabels=class_names)
plt.title(f"{model_name} – Testing CM")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.show()

# Save figure
plt.savefig(f"/content/drive/MyDrive/{model_name}_confusion_matrix.png")


**Cell 9: Plot ROC Curve**

In [ ]:
fpr_train, tpr_train, _ = roc_curve(y_train_true, y_train_prob[:,1])
fpr_test,  tpr_test,  _ = roc_curve(y_test_true,  y_test_prob[:,1])

auc_train = auc(fpr_train, tpr_train)
auc_test  = auc(fpr_test, tpr_test)

plt.figure(figsize=(6,5))
plt.plot(fpr_train, tpr_train, label=f"Train AUC = {auc_train:.3f}")
plt.plot(fpr_test,  tpr_test,  label=f"Test AUC = {auc_test:.3f}")
plt.plot([0,1], [0,1], 'k--')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title(f"{model_name} ROC Curve")
plt.legend()
plt.grid()
plt.show()

# Save figure
plt.savefig(f"/content/drive/MyDrive/{model_name}_roc_curve.png")
